In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
1,146,SI,VIGENTE,TRÁNSITO RESTRINGIDO,11/01/2023,25/01/2023 18:00 HORAS,CUSCO/LA CONVENCION/SANTA ANA,KM 199,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.873069617394,...,-12.873069617394+-72.692213162262,2023 18:00 HORAS+01+25+18:00,0,0,COE-MTC,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
2,147,SI,VIGENTE,TRÁNSITO RESTRINGIDO,07/01/2023,25/01/2023 18:00 HORAS,CUSCO/LA CONVENCION/SANTA ANA,KM 230,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.880884243624,...,-12.880884243624+-72.690784623809,2023 18:00 HORAS+01+25+18:00,0,1,COE-MTC,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
3,161,SI,VIGENTE,TRÁNSITO RESTRINGIDO,19/01/2023,25/01/2023 18:00 HORAS,CUSCO/URUBAMBA/MARAS,KM 44,PUENTE URUBAMBA. PARO INDEFINIDO SOLICITANDO N...,-13.312531099811,...,-13.312531099811+-72.117755850561,2023 18:00 HORAS+01+25+18:00,0,0,COE-MTC,HUMANO,PE-28F,CARRETERA POROY - CHINCHERO - URUBAMBA,SI,restringido
4,137,SI,VIGENTE,TRÁNSITO RESTRINGIDO,10/01/2023,25/01/2023 18:00 HORAS,CUSCO/ESPINAR/ESPINAR,KM 245,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-14.794379646982,...,-14.794379646982+-71.409019289269,2023 18:00 HORAS+01+25+18:00,0,0,COE-MTC,HUMANO,PE-3SG,CARRETERA CHALHUAHUACHO - ESPINAR - AYAVIRI,SI,restringido
5,73,SI,VIGENTE,TRÁNSITO RESTRINGIDO,23/01/2023,25/01/2023 18:00 HORAS,AREQUIPA/AREQUIPA/YURA,KM 51,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.2946789,...,-16.2946789+-71.6313608,2023 18:00 HORAS+01+25+18:00,0,1,UD AREQUIPA,HUMANO,PE-34A,CARRETERA REPARTICION AREQUIPA - SANTA LUCIA -...,SI,restringido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,335,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,25/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/IRAZOLA,KM 380,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-8.9382,...,-8.9382+-75.3882,2023 18:00 HORAS+01+25+18:00,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
97,338,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,21/01/2023,25/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 390,(SECTOR HUIPOCA) PARO INDEFINIDO SOLICITANDO N...,-8.976077,...,-8.976077+-75.44483,2023 18:00 HORAS+01+25+18:00,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
98,339,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,25/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 406,(SECTOR BARRIO UNIDO -AGUAYTIA) PARO INDEFINID...,-9.051327386,...,-9.051327386+-75.5166082,2023 18:00 HORAS+01+25+18:00,0,0,COE-MTC,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
99,340,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,25/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 419,(SECTOR BOQUERON) PARO INDEFINIDO SOLICITANDO ...,-9.057925989,...,-9.057925989+-75.633071,2023 18:00 HORAS+01+25+18:00,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
